In [ ]:
## In Class 2021-10-18

In [20]:
## Q1 Using the pandas library, read the csv data file and create a data-frame called heart.

import pandas as pd
import boto3
import numpy as np
## import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
## for regression is RandomForestRegressor
## from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.metrics import recall_score


## Defining the S3 bucket
s3 = boto3.resource('s3')
bucket_name = 'bonnieh-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##R Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [21]:
## Q2. Remove observations with missing values.

heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [22]:
## Q3 Using age, currentSmoker, totChol, BMI, and heartRate as the predictor variables, and
## TenYearCHD is the target variable, split the data into two data-frames (taking into account
## the proportion of 0s and 1s): train (80%) and test (20%).

## Defining the variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify = Y)
X_test.head()

,age,currentSmoker,totChol,BMI,heartRate
224,43,0,257.0,21.95,100.0
1057,41,1,176.0,24.04,78.0
3983,41,1,188.0,28.60,85.0
430,40,0,195.0,22.36,70.0
650,58,1,281.0,22.54,58.0


In [23]:
## Q4. Transform each of the input variables in the train and test dataset to 0-1 scale.

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

X_test

array([[0.25714286, 0.        , 0.42352941, 0.1878969 , 0.69135802],
       [0.2       , 1.        , 0.18529412, 0.26596937, 0.41975309],
       [0.2       , 1.        , 0.22058824, 0.4363093 , 0.50617284],
       ...,
       [0.28571429, 1.        , 0.28823529, 0.24168846, 0.19753086],
       [0.74285714, 0.        , 0.33235294, 0.52035861, 0.44444444],
       [0.65714286, 1.        , 0.28529412, 0.38177064, 0.40740741]])

In [27]:
## Q5.  Using the train data-frame, build an adaboost model (with 500 trees and max tree depth
## equal 3) in which: age, currentSmoker, totChol, BMI, and heartRate are the predictor variables,
## and TenYearCHD is the target variable. Using this model, predict the risk of coronary disease of the
## patients in the test data-frame. Using 10% as cutoff value, report the recall.## Adaboost model
## md1 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)

## making predictions on the test dataset
pred1 = md1.predict_proba(X_test)[:,1]
pred1 = np.where(pred1 < 0.1, 0, 1)

## Compute the recal

recall_score(Y_test, pred1)

1.0

In [28]:
## Q6Using the train data-frame, build a random forest model (with 500 trees and max tree depth
## equal 3) in which: age, currentSmoker, totChol, BMI, and heartRate are the predictor variables,
## and TenYearCHD is the target variable. Using this model, predict the risk of coronary disease of the
## patients in the test data-frame. Using 10% as cutoff value, report the recall.

## Random forest

md2 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

## making predictions on the test dataset
pred2 = md2.predict_proba(X_test)[:,1]
pred2 = np.where(pred2 < 0.1, 0, 1)

## Compute the recall

recall_score(Y_test, pred2)

0.9910714285714286

In [ ]:
## Using the results from parts (5) and (6), what model would you use to predict TenYearCHD
## based on the recall?

## based on the recall values, I would use the AdaBoost model